# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
ls *

event_datafile_new.csv  Project_1B_ Project_Template.ipynb

event_data:
2018-11-01-events.csv  2018-11-11-events.csv  2018-11-21-events.csv
2018-11-02-events.csv  2018-11-12-events.csv  2018-11-22-events.csv
2018-11-03-events.csv  2018-11-13-events.csv  2018-11-23-events.csv
2018-11-04-events.csv  2018-11-14-events.csv  2018-11-24-events.csv
2018-11-05-events.csv  2018-11-15-events.csv  2018-11-25-events.csv
2018-11-06-events.csv  2018-11-16-events.csv  2018-11-26-events.csv
2018-11-07-events.csv  2018-11-17-events.csv  2018-11-27-events.csv
2018-11-08-events.csv  2018-11-18-events.csv  2018-11-28-events.csv
2018-11-09-events.csv  2018-11-19-events.csv  2018-11-29-events.csv
2018-11-10-events.csv  2018-11-20-events.csv  2018-11-30-events.csv

images:
image_event_datafile_new.jpg


In [3]:
print(os.getcwd())
filepath = os.getcwd() + '/event_data'


for root, dirs, files in os.walk(filepath):
    
    file_path_list = glob.glob(os.path.join(root,'../*'))
    print(len(file_path_list))

/home/workspace
4
30


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
full_data_rows_list = [] 
    
 
for f in file_path_list:

 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
       
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
 
print(len(full_data_rows_list))

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [5]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


In [6]:
full_data_rows_list[1]

['Manowar',
 'Logged In',
 'Jacob',
 'M',
 '0',
 'Klein',
 '247.562',
 'paid',
 'Tampa-St. Petersburg-Clearwater, FL',
 'PUT',
 'NextSong',
 '1.54056E+12',
 '1049',
 'Shell Shock',
 '200',
 '1.54354E+12',
 '73']

# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [7]:
from cassandra.cluster import Cluster
cluster = Cluster()


session = cluster.connect()

#### Create Keyspace

In [8]:
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS project 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

#### Set Keyspace

In [9]:
session.set_keyspace('project')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [10]:
query = "CREATE TABLE IF NOT EXISTS song_playlist_item_session"
query += '''(
             session_id int,
             item_in_session int,
             artist_name text,
             song_length float,
             song_title text,
             PRIMARY KEY (session_id, item_in_session)
             )'''
try:
    session.execute(query)
except Exception as e:
    print(e)                   

In [11]:

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:

        query = "INSERT INTO song_playlist_item_session (session_id, item_in_session, artist_name, song_length, song_title)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), str(line[0]), float(line[5]), str(line[9])))
        #print(line)
       
        

#### Do a SELECT to verify that the data have been inserted into each table

In [12]:

# query1 show the artist, song title and song's length in the music app history that was heard during  
# sessionId = 338, and itemInSession  = 4

query = """
SELECT 

     artist_name,
     song_length,
     song_title
     
FROM 
     song_playlist_item_session
WHERE 
     session_id= 338 
AND 
     item_in_session= 4
     
"""
 
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.artist_name , row.song_length, row.song_title )
    


Faithless 495.30731201171875 Music Matters (Mark Knight Dub)


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [13]:
query = "CREATE TABLE IF NOT EXISTS song_playlist_session"
query += '''(
             session_id int,
             user_id int,
             item_in_session int,
             artist_name text,
             song_title text,
             first_name text,
             last_name text,
             PRIMARY KEY ((user_id , session_id), item_in_session)
             )'''
try:
    session.execute(query)
except Exception as e:
    print(e)  




                    

In [14]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:

        query = "INSERT INTO song_playlist_session (session_id, user_id, item_in_session, artist_name, song_title, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[10]), int(line[3]), str(line[0]), str(line[9]), str(line[1]), str(line[4]))) 

        

In [15]:
# query2 show the name of artist, song (sorted by itemInSession) and user (first and last name) 
#     for userid = 10, sessionid = 182
query ="""
SELECT
    
   artist_name,
   song_title,
   first_name,
   last_name
FROM 
    song_playlist_session

WHERE 
    user_id= 10
AND 
   session_id= 182 
   
ORDER BY
   item_in_session
    
"""
                           
try:
    rows = session.execute(query)
                           
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist_name , row.song_title, row.first_name, row.last_name)
    

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [16]:
query = "CREATE TABLE IF NOT EXISTS song_playlist_library"
query += '''(
        song_title text,
        user_id int,
        first_name text,
        last_name text,
        PRIMARY KEY (song_title, user_id)
             )'''
try:
    session.execute(query)
except Exception as e:
    print(e)  

    
    
                    

In [17]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query = "INSERT INTO song_playlist_library (song_title, user_id, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))


In [18]:
#query3 show every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query ="""
SELECT
    
    song_title,
    first_name,
    last_name
FROM 
    song_playlist_library

WHERE 
    song_title ='All Hands Against His Own'
    
"""

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.first_name , row.last_name , row.song_title)


Jacqueline Lynch All Hands Against His Own
Tegan Levine All Hands Against His Own
Sara Johnson All Hands Against His Own


### Drop the tables before closing out the sessions

In [19]:
query = "drop table song_playlist_item_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table song_playlist_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table song_playlist_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [20]:
session.shutdown()
cluster.shutdown()